In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn import preprocessing
import time
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.metrics.pairwise import euclidean_distances
%matplotlib inline

In [4]:
industry_df = pd.read_csv('Data/movie_industry.csv', encoding='latin-1')
oscar_df = pd.read_csv('Data/the_oscar_award.csv', encoding='latin-1')
links_df = pd.read_csv('Data/movie_lense/links.csv')
tags_df = pd.read_csv('Data/movie_lense/tags.csv')
genome_scores_df = pd.read_csv('Data/movie_lense/genome-scores.csv')
movies_df = pd.read_csv('Data/movie_lense/movies.csv')
ratings_df = pd.read_csv('Data/movie_lense/ratings.csv')
genome_tags_df = pd.read_csv('Data/movie_lense/genome-tags.csv')

In [5]:
ratings_df
userID=5
def extract_user_movies(userID,thresh=4):
    tdf = ratings_df[ratings_df['userId']==userID]
    tdf = tdf[tdf['rating']>=thresh]
    return tdf['movieId'].values

extract_user_movies(5)

array([   47,    50,   296,   318,   527,   778,   858,  1147,  1193,
        1213,  1222,  1617,  1732,  2329,  2502,  2858,  2959,  3178,
        3949,  4011,  4226,  4973,  5110,  5137,  5954,  5995,  6016,
        7361,  8784,  8873,  8949,  8950, 27831, 30749, 30820, 31410,
       32587, 33794, 38061, 40278, 44195, 44204, 46976, 48394, 48516,
       48696, 48774, 48780, 49272, 49530, 50068, 51255, 52952, 54286,
       54503, 54997, 55118, 55247, 55290, 55765, 55820, 56367, 56782],
      dtype=int64)

In [25]:
tgsdf = genome_scores_df.pivot(index='movieId',columns='tagId',values='relevance')
genome_tags_dict = dict(zip(genome_tags_df['tagId'],genome_tags_df['tag']))
movie_names_dict = dict(zip(movies_df['movieId'],movies_df['title']))
tags = genome_tags_df['tag'].values
# genome_tags_dict
# movie_names_dict
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [19]:
movie_tags_df = tgsdf.rename(genome_tags_dict,axis='columns')
movie_tags_thresh_df = movie_tags_df.copy()
thresh = 0.8
movie_tags_thresh_df[movie_tags_thresh_df<thresh] = 0
movie_tags_thresh_df[movie_tags_thresh_df>thresh] = 1
# movie_tags_df.sum()
movie_tags_df

tagId,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,0.02400,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,0.05250,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,0.03200,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,0.02400,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,0.02375,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185435,0.10625,0.10775,0.03025,0.05650,0.23175,0.04575,0.05175,0.44700,0.19875,0.04025,...,0.34725,0.03125,0.03500,0.12125,0.32200,0.08200,0.03325,0.02425,0.13725,0.02625
185585,0.04250,0.04175,0.06475,0.07625,0.08500,0.03800,0.03425,0.16850,0.02525,0.03525,...,0.04325,0.05850,0.03050,0.02750,0.21850,0.04525,0.03550,0.02550,0.17475,0.03100
186587,0.08950,0.09125,0.05075,0.04900,0.19475,0.04500,0.03125,0.17275,0.07850,0.02475,...,0.06475,0.04500,0.04000,0.01975,0.22075,0.15125,0.06150,0.04300,0.20800,0.03925


In [32]:
# Make matrix of euclidian distance b/w each movie
dist = euclidean_distances(movie_tags_df,movie_tags_df)

In [18]:
movie_tags_df.iloc[0,1]

0.023749999999999997

In [33]:
np.fill_diagonal(dist,np.median(dist)) # Make all diags median so they doont influence min or max
dist

array([[5.80485434, 5.60443644, 6.33694617, ..., 6.67146577, 6.76831775,
        5.63426966],
       [5.60443644, 5.80485434, 4.9950365 , ..., 5.46353316, 7.32374549,
        5.24566639],
       [6.33694617, 4.9950365 , 5.80485434, ..., 5.31612765, 6.81545049,
        5.03226821],
       ...,
       [6.67146577, 5.46353316, 5.31612765, ..., 5.80485434, 6.3338389 ,
        5.2941725 ],
       [6.76831775, 7.32374549, 6.81545049, ..., 6.3338389 , 5.80485434,
        5.05494847],
       [5.63426966, 5.24566639, 5.03226821, ..., 5.2941725 , 5.05494847,
        5.80485434]])

In [35]:
x, y = np.unravel_index(dist.argmin(), dist.shape)

In [45]:
print(movies_df[movies_df['movieId']==movie_tags_df.index[x]])
print(movies_df[movies_df['movieId']==movie_tags_df.index[y]])

       movieId         title                 genres
13173    63436  Saw V (2008)  Crime|Horror|Thriller
       movieId          title                         genres
14456    72129  Saw VI (2009)  Crime|Horror|Mystery|Thriller
